In [450]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [451]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
ss = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [452]:
print(train.shape)
print(test.shape)
print(ss.shape)

In [453]:
print(train.isnull().sum())
print(test.isnull().sum())

In [454]:
import string
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
!pip install geotext

In [455]:
from geotext import GeoText

In [456]:
train["num_punctuations"] =train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test['num_punctuations'] = test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

In [457]:
f, ax = plt.subplots(figsize=(10, 5))
sns.kdeplot(x = train['num_punctuations'], hue = train['target'])

In [458]:
train['text_len'] = train['text'].str.len()
test['text_len'] = test['text'].str.len()

In [459]:
f, ax = plt.subplots(figsize =(10, 5))
sns.kdeplot(train['text_len'], hue=train['target'])

In [460]:
train['hash_count'] = train['text'].str.count('#')
test['hash_count'] = test['text'].str.count('#')

In [461]:
f, ax = plt.subplots(figsize = (10, 5))
sns.countplot(x = train['hash_count'], hue = train['target'])

In [462]:
f, ax = plt.subplots(figsize =(10, 5))
sns.kdeplot(train['hash_count'], hue=train['target'])

In [463]:
f, ax = plt.subplots(figsize =(10, 5))
sns.countplot(x = train['keyword'], hue = train['target'], order=train['keyword'].value_counts().iloc[:10].index)

In [464]:
f, ax = plt.subplots(figsize=(10, 5))
sns.countplot(x = train['location'], hue=train['target'], order=train['location'].value_counts().iloc[:10].index)

In [465]:
print(train[train['keyword'].isnull()]['text'].str.contains('#').value_counts())
print('\n')
print(train[train['location'].isnull()]['text'].str.contains('#').value_counts())
print('\n')
print(train[(train['location'].isnull()) & (train['keyword'].isnull())]['text'].str.contains('#').value_counts())

In [466]:
def hash_words(data):
    ls = []
    for idx in range(0, len(data)):
        temp_ls = []
        hashtags = []
        temp_ls = data['text'].iloc[idx].split()
        for i in temp_ls:
            if '#' in i:
                i = i.strip('#')
                hashtags.append(i)
        ls.append(hashtags)
    return ls

In [467]:
hash_ls = hash_words(train)
hash_ls1 = hash_words(test)

In [468]:
def make_set(data):
    loc_set = []
    key_set = []
    for i in range(0, len(data)):
        loc_set.append(data['location'].iloc[i])
        key_set.append(data['keyword'].iloc[i])

    set_loc = set(loc_set)
    set_key = set(key_set)
    set_loc.remove(np.nan)
    set_key.remove(np.nan)
    
    return set_loc, set_key

In [469]:
# def test_make_set(data):
#     loc_set = []
#     key_set = []
#     for i in range(0, len(data)):
#         loc_set.append(data['location'].iloc[i])
#         key_set.append(data['keyword'].iloc[i])

#     set_loc = set(loc_set)
#     set_key = set(key_set)
#     set_loc.remove('NAN')
#     set_key.remove('NAN')
    
#     return set_loc, set_key

In [470]:
train_loc, train_key = make_set(train)
test_loc, test_key = make_set(test)

In [471]:
train_test = train.copy(deep=True)
test_test = test.copy(deep=True)

In [472]:
train_test['location'].fillna('NAN', inplace=True)
train_test['keyword'].fillna('NAN', inplace=True)

test_test['location'].fillna('NAN', inplace=True)
test_test['keyword'].fillna('NAN', inplace=True)

In [473]:
play = 0
for i in range(0, len(hash_ls1)):
    for j in range(0, len(hash_ls1[i])):
        if (hash_ls1[i][j] in test_loc) & (test_test['location'].iloc[i] == 'NAN'):
            test_test.iloc[i, test_test.columns.get_loc('location')] = hash_ls1[i][j]
            play += 1
            continue

print(play)

In [474]:
play = 0
for i in range(0, len(hash_ls)):
    for j in range(0, len(hash_ls[i])):
        if (hash_ls[i][j] in train_key) & (train_test['keyword'].iloc[i] == 'NAN'):
            train_test.iloc[i, train_test.columns.get_loc('keyword')] = hash_ls[i][j]
            play += 1
            continue

print(play)

In [475]:
play = 0
for i in range(0, len(hash_ls)):
    for j in range(0, len(hash_ls[i])):
        if (hash_ls[i][j] in train_loc) & (train_test['location'].iloc[i] == 'NAN'):
            train_test.iloc[i, train_test.columns.get_loc('location')] = hash_ls[i][j]
            play += 1
            continue

print(play)

In [476]:
play = 0
for i in range(0, len(hash_ls1)):
    for j in range(0, len(hash_ls1[i])):
        if (hash_ls1[i][j] in test_key) & (test_test['keyword'].iloc[i] == 'NAN'):
            test_test.iloc[i, test_test.columns.get_loc('keyword')] = hash_ls1[i][j]
            play += 1
            continue

print(play)

In [477]:
for i in range(0, 10):
    print(train_test['text'].iloc[i], train_test['target'].iloc[i], train_test['location'].iloc[i], train_test['keyword'].iloc[i])

In [478]:
eng_stopwords = set(stopwords.words('english'))

In [479]:
from nltk.tokenize import TweetTokenizer
tokeniz = TweetTokenizer()

sentences = []
for i in range(0, len(train)):
    token = tokeniz.tokenize(train['text'].iloc[i])
    filtered_sentence = [w for w in token if not w.lower() in eng_stopwords]
    sentences.append(filtered_sentence)

In [480]:
sentences1 = []
for i in range(0, len(test)):
    token = tokeniz.tokenize(test['text'].iloc[i])
    filtered_sentence = [w for w in token if not w.lower() in eng_stopwords]
    sentences1.append(filtered_sentence)

In [481]:
for i in range(0, 10):
    print(sentences[i])

from geotext import GeoText

places = GeoText("London is a great city")

places.cities -> 'london' 이렇게 나오고

GeoText('New York, Texas, and also China').country_mentions

-> OrderedDict() 형태로 나옴

In [482]:
play = 0
for i in range(0, len(sentences)):
    if train_test['location'].iloc[i] == 'NAN':
        geo_sent = GeoText(','.join(sentences[i]))
        #print(geo_sent.cities)
        #print(geo_sent.country_mentions)
        if len(geo_sent.cities) != 0 or geo_sent.country_mentions :
            play+=1
            
print(play)

2469개 결측치중에서 523개를 채울 수 있다고 나옴

그럼 한번 채워보도록 하겠음

In [483]:
play = 0
for i in range(0, len(sentences1)):
    if test_test['location'].iloc[i] == 'NAN':
        geo_sent = GeoText(','.join(sentences[i]))
        #print(geo_sent.cities)
        #print(geo_sent.country_mentions)
        if len(geo_sent.cities) != 0 or geo_sent.country_mentions :
            play+=1
            
print(play)

In [484]:
city = 0
nation = 0

for i in range(0, len(sentences)):
    if train_test['location'].iloc[i] == 'NAN':
        geo_sent = GeoText(','.join(sentences[i]))
        #print(geo_sent.cities)
        #print(geo_sent.country_mentions)
        if len(geo_sent.cities) != 0:
            city +=1
        if geo_sent.country_mentions:
            nation += 1
print(city)
print(nation)

오바마씨가 진짜 짜증나게 했다... 오바마시라는 일본의 시가 있어서 JP로 인식되는것이 유일한 단점이었다.

-> 이거는 처리를 해주는 것이 좋을 것 같았지만 머리가 깨질 거 같다 Obama씨가 등장함에 따라 target이 0인지 1인지 극명하게 갈릴테니 그냥 냅두기로 했다.

도시만 언급한 경우가 313개 -> 자동으로 나라가 특정됨

나라만 언급한 경우가 210개


In [485]:
print(train_test['location'].value_counts())

for i in range(0, len(sentences)):
    if train_test['location'].iloc[i] == 'NAN':
        geo_sent = GeoText(','.join(sentences[i]))
        #print(geo_sent.cities)
        #print(geo_sent.country_mentions)
        if len(geo_sent.country_mentions) > 0:
            if len(geo_sent.country_mentions) == 1:
                if len(geo_sent.cities) == 0:
                    train_test.iloc[i, train_test.columns.get_loc('location')] = list(geo_sent.country_mentions)[0]
                elif len(geo_sent.cities) == 1:
                    train_test.iloc[i, train_test.columns.get_loc('location')] = geo_sent.cities[0]
                else:
                    temp = set(geo_sent.cities)
                    if len(temp) == 1:
                        train_test.iloc[i, train_test.columns.get_loc('location')] = list(temp)[0]
                    else:
                        train_test.iloc[i, train_test.columns.get_loc('location')] = list(geo_sent.country_mentions)[0]
            else:
                if len(geo_sent.cities) == 0:
                    train_test.iloc[i, train_test.columns.get_loc('location')] = list(geo_sent.country_mentions)[0]
                elif len(geo_sent.cities) == 1:
                    train_test.iloc[i, train_test.columns.get_loc('location')] = geo_sent.cities[0]
                else:
                    train_test.iloc[i, train_test.columns.get_loc('location')] = list(geo_sent.country_mentions)[0]
                    
print(train_test['location'].value_counts())

나라가 1개만 특정될경우 도시의 특정여부에 따라

도시가 특정될경우 도시이름으로 대체해주고 도시가 특정되지않을 경우 나라이름으로 특정해주면 될것 같다

523개의 결측값이 대체되었다. (야무지게 대체해줌)

In [486]:
print(test_test['location'].value_counts())

for i in range(0, len(sentences1)):
    if test_test['location'].iloc[i] == 'NAN':
        geo_sent = GeoText(','.join(sentences1[i]))
        #print(geo_sent.cities)
        #print(geo_sent.country_mentions)
        if len(geo_sent.country_mentions) > 0:
            if len(geo_sent.country_mentions) == 1:
                if len(geo_sent.cities) == 0:
                    test_test.iloc[i, test_test.columns.get_loc('location')] = list(geo_sent.country_mentions)[0]
                elif len(geo_sent.cities) == 1:
                    test_test.iloc[i, test_test.columns.get_loc('location')] = geo_sent.cities[0]
                else:
                    temp = set(geo_sent.cities)
                    if len(temp) == 1:
                        test_test.iloc[i, test_test.columns.get_loc('location')] = list(temp)[0]
                    else:
                        test_test.iloc[i, test_test.columns.get_loc('location')] = list(geo_sent.country_mentions)[0]
            else:
                if len(geo_sent.cities) == 0:
                    test_test.iloc[i, train_test.columns.get_loc('location')] = list(geo_sent.country_mentions)[0]
                elif len(geo_sent.cities) == 1:
                    test_test.iloc[i, test_test.columns.get_loc('location')] = geo_sent.cities[0]
                else:
                    test_test.iloc[i, test_test.columns.get_loc('location')] = list(geo_sent.country_mentions)[0]
                    
print(test_test['location'].value_counts())

In [487]:
# translator = Translator()
# translator.translate('안녕하세요.')

In [488]:
test_sentences = GeoText('RockyMountain')
print(test_sentences.cities)
print(test_sentences.country_mentions)

In [489]:
import re
string.punctuation

In [490]:
ex_words = '['+string.punctuation+']'
print(ex_words)

In [491]:
ex_words = ex_words[0:23] + ex_words[24] + ex_words[26:]

print(ex_words)

In [492]:
ex_words1 = ex_words[:3] + ex_words[4:]
print(ex_words1)

In [493]:
train_test['parse_text'] = train['text']
test_test['parse_text'] = test['text']

In [494]:
train_test['parse_text'] = train['text'].apply(lambda x: str(re.sub(ex_words, '', x)))
test_test['parse_text'] = test['text'].apply(lambda x: str(re.sub(ex_words, '', x)))

In [495]:
for i in range(0, 10):
    print(train_test['parse_text'].iloc[i])

In [496]:
for i in range(0, 10):
    print(train_test['text'].iloc[i])

In [497]:
train_test["num_words_title"] = train_test["parse_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test_test["num_words_title"] = test_test["parse_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

In [498]:
f, ax = plt.subplots(figsize =(10, 5))
sns.countplot(x = train_test['num_words_title'], hue = train_test['target'])

title_word의 수가 4개 이상 포함되어있는 Text부터 재난이 일어난 트윗의 수가 일어나지 않은 수보다 많아졌다.

text를 분석해보니 hypertext만 다르고 text 내용이 판박이인 친구들이 꽤나 있었다 -> 이것도 활용하면 좋을거 같은데;;

In [499]:
train_test["num_words_upper"] = train_test["parse_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test_test['num_words_upper'] = test_test['parse_text'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

In [500]:
f, ax = plt.subplots(figsize=(10, 5))
sns.countplot(x = train_test['num_words_upper'], hue = train_test['target'])

In [501]:
train_test['hash_count'] = train_test['hash_count'] + 1
train_test['num_words_upper'] = train_test['num_words_upper'] + 1

test_test['hash_count'] = train_test['hash_count'].apply(lambda x : np.log(x))
test_test['num_words_upper'] = train_test['num_words_upper'].apply(lambda x : np.log(x))


test_test['hash_count'] = test_test['hash_count'] + 1
test_test['num_words_upper'] = test_test['num_words_upper'] + 1

test_test['hash_count'] = test_test['hash_count'].apply(lambda x : np.log(x))
test_test['num_words_upper'] = test_test['num_words_upper'].apply(lambda x : np.log(x))

In [502]:
train_test = pd.get_dummies(data = train_test, columns = ['keyword'], prefix = 'key_')
test_test = pd.get_dummies(data = test_test, columns = ['keyword'], prefix = 'key_')

In [503]:
features = []
features1 = []
for col in train_test:
    features.append(col)
    
for col in test_test:
    features1.append(col)

In [504]:
rem = ['id', 'location', 'text', 'target', 'parse_text']
rem1 = ['id', 'location', 'text', 'parse_text']
for col in rem:
    features.remove(col)
    
for col in rem1:
    features1.remove(col)

In [505]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
K_fold = KFold(n_splits=10, shuffle=True, random_state=0)

y = train_test['target'].values
X = train_test[features].values
    

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [506]:
X_train.shape

In [507]:
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('Default RandomForestClassifier acc : {:.4f}'.format(accuracy))

clf2 = RandomForestClassifier(n_estimators = 10)
scoring = 'accuracy'
score2 = cross_val_score(clf2, X, y, cv=K_fold, n_jobs=1, scoring=scoring)

round(np.mean(score2) * 100, 2)

In [509]:
ss['target'] = rfc.predict(test_test[features].values)

In [510]:
ss.to_csv('test_submission.csv', index=False)

In [ ]:
train_test['clean_text'] = train_test['text'].str.replace("[^a-zA-Z]", " ")

In [ ]:
train_test['lower_clean_text'] = train_test['clean_text'].apply(lambda x: x.lower())

In [ ]:
train_test['lower_clean_text']

In [ ]:
tokenized_text = train_test['lower_clean_text'].apply(lambda x : x.split())
tokenized_text = tokenized_text.apply(lambda x: [item for item in x if item not in eng_stopwords])

In [ ]:
text_ls = []
for i in range(0, len(train_test)):
    t = ' '.join(tokenized_text[i])
    text_ls.append(t)

train_test['clean_text'] = text_ls

In [ ]:
train_test['clean_text']

In [ ]:
train_test['remove_https_text'] = train_test['clean_text']

In [ ]:
for i in range(0, len(train_test)):
    if train_test['clean_text'].iloc[i].find('http') != -1:
        find_loc = train_test['clean_text'].iloc[i].find('http')
        train_test['remove_https_text'].iloc[i] = (train_test['clean_text'].iloc[i])[0:find_loc]
        #train_test.iloc[i, train_test.columns.get_loc('remove_https_text')] = string(train_test['clean_text'].iloc[i])[0:find_loc]

http가 있는 TEXT에 대해서 http 처음 나온 위치에서 시작하여 그 뒤 TEXT를 싹다 지우기

LSA 진행시 http 때문에 단어의 빈출도가 이상해지는 경향이 있어서 싹다 지워버림

In [ ]:
train_test['remove_https_text'].value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(train_test['clean_text'])
print(X.shape)

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
print(len(svd_model.components_))
print(np.shape(svd_model.components_))

In [ ]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer1 = TfidfVectorizer(stop_words='english', 
max_features= 700, # 상위 700개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X1 = vectorizer1.fit_transform(train_test['remove_https_text'])
print(X1.shape)

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd_model1 = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
svd_model1.fit(X1)
print(len(svd_model1.components_))

In [ ]:
terms1 = vectorizer1.get_feature_names() # 단어 집합. 700개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model1.components_,terms1)